In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import random
import json
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Download NLTK data
nltk.download('punkt')

stemmer = LancasterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

# Load data
with open('/intents.json') as f:
    data = json.load(f)

In [ ]:

# Data preprocessing
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(" ".join(wrds))
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [ ]:
# Stemming and lowercasing the words
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [ ]:

# Define the tokenizer function
def tokenize(text):
    return nltk.word_tokenize(text.lower())

In [ ]:

# Vectorize and train the model
vectorizer = TfidfVectorizer(tokenizer=tokenize)
model = make_pipeline(vectorizer, MultinomialNB())

model.fit(docs_x, docs_y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x78c7e46ebb50>)),
                ('multinomialnb', MultinomialNB())])

In [ ]:

# Save the model
with open("chatbot_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

In [ ]:

# Function to predict response
def chat():
    with open("chatbot_model.pkl", "rb") as model_file:
        model = pickle.load(model_file)

    with open("training_data.pkl", "rb") as data_file:
        data = pickle.load(data_file)
        words = data['words']
        labels = data['labels']
        intents = data['intents']

    print("\033[1;31mStart talking with the bot! (Type 'quit' to stop)")
    while True:
        inp = input("\033[1;31mYou: ")
        if inp.lower() == "quit":
            break

        results = model.predict([inp])
        tag = results[0]

        for tg in intents:
            if tg['tag'] == tag:
                responses = tg['responses']
                response_list = nltk.sent_tokenize(str(responses[0]))
                colors = [31, 32, 33, 34, 35]

                for i, response in enumerate(response_list):
                    color_index = i % len(colors)
                    print(f'\033[1;{colors[color_index]}m{response}\n')



In [ ]:
chat()


Start talking with the bot! (Type 'quit' to stop)
You: fever
To treat a fever at home: 1)Drink plenty of fluids to stay hydrated.

2)Dress in lightweight clothing.

3)Use a light blanket if you feel chilled, until the chills end.

4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others).

5) Get medical help if the fever lasts more than five days in a row.

You: quit
